In [ ]:
import numpy as np
import graphinglib as gl
import uncertainties

from typing import Self

In [ ]:
class Micrograph:
    def __init__(self, data: np.ndarray, index: int) -> Self:
        self.data = data
        self._index = index
        self.fft = np.fft.fftshift(np.fft.fft2(data))
        self.image_center = np.array(self.data.shape) / 2

    @classmethod
    def load(cls, filename: str) -> Self:
        return cls(gl.Heatmap(filename).image, int(filename[-5]))
    
    @property
    def data_plot(self) -> gl.Heatmap:
        return gl.Heatmap(self.data, aspect_ratio="equal", color_map="gray", show_color_bar=False)

    @property
    def fft_plot(self) -> gl.Heatmap:
        return gl.Heatmap(np.abs(self.fft), aspect_ratio="equal", vmax=1e5, color_map="gray", show_color_bar=False)

    @property
    def node_coordinates(self) -> np.ndarray:
        match self._index:
            case 1:
                return np.array([
                    [524, 844],
                    [642, 555],
                ])
            case 2:
                return np.array([
                    [512.5, 734.5],
                    [628, 477.5],
                ])
            case 3:
                return np.array([
                    [594, 747.5],
                    [648.5, 526],
                ])
            case _:
                raise ValueError("Unsupported index")

    @property
    def vectors(self) -> np.ndarray:
        return self.node_coordinates - self.image_center[0]

    @property
    def vector_lengths(self) -> tuple[uncertainties.core.Variable, uncertainties.core.Variable]:
        return (
            uncertainties.ufloat(np.linalg.norm(self.node_coordinates[0]-512), 3),
            uncertainties.ufloat(np.linalg.norm(self.node_coordinates[1]-512), 3)
        )
    
    @property
    def angle_between_vectors(self) -> float:
        return self.get_angle_between_vectors(self.vectors[0], self.vectors[1])

    @property
    def arrows(self) -> list[gl.Arrow, gl.Text, gl.Arrow, gl.Text]:
        n_coords = self.node_coordinates
        c = self.image_center
        return [
            gl.Arrow(c, (n_coords[0,0], n_coords[0,1]), color="yellow", width=2),
            gl.Text((n_coords[0,0]-c[0])/2+c[0]+25, (n_coords[0,1]-c[1])/2+c[1], "h", color="yellow"),
            gl.Arrow(c, (n_coords[1,0], n_coords[1,1]), color="lime", width=2),
            gl.Text((n_coords[1,0]-c[0])/2+c[0], (n_coords[1,1]-c[1])/2+c[1]-25, "k", color="lime"),
        ]
    
    @property
    def plane_distances(self) -> dict[str, float]:
        return {
            "100" : np.sin(self.angle_between_vectors)*self.vector_lengths[0],
            "010" : np.sin(self.angle_between_vectors)*self.vector_lengths[1],
            "110" : np.sin(self.get_angle_between_vectors(self.vectors[1], self.vectors[1]-self.vectors[0]))
                    * self.vector_lengths[1],
            "1-10" : np.sin(self.get_angle_between_vectors(self.vectors[1], self.vectors[1]+self.vectors[0]))
                     * self.vector_lengths[1],
        }
    
    @staticmethod
    def get_angle_between_vectors(vector_1: np.ndarray, vector_2: np.ndarray) -> float:
        return np.arccos(np.dot(vector_1, vector_2) / (np.linalg.norm(vector_1) * np.linalg.norm(vector_2)))


In [ ]:
# %matplotlib tk
%matplotlib inline

figs = []
for i in range(1,4):
    mg = Micrograph.load(f"design/data/Micrographie_{i}.tif")
    fig = gl.Figure(size=(5, 5), figure_style="dark", title=f"Micrographie {i}", remove_axes=True)
    # fig.add_elements(mg.data_plot)
    fig.add_elements(mg.fft_plot, *mg.arrows)
    figs.append(fig)
    # fig.show()

multifig = gl.MultiFigure.from_row(figs, size=(15, 5), figure_style="dark")
multifig.show()

In [ ]:
im_size = 1024
px_res = 42.468     # picometers per pixel

for i in range(1,4):
    mg = Micrograph.load(f"design/data/Micrographie_{i}.tif")

    for key, value in mg.plane_distances.items():
        print(f"Micrographie {i}, {key} : {im_size/(value)*px_res} pm")
    print()

### Plane figure

In [ ]:
mg = Micrograph.load(f"design/data/Micrographie_1.tif")

fig = gl.Figure(size=(6,6), figure_style="dark", remove_axes=True)

h = lambda n: mg.image_center + mg.vectors[0]*n
k = lambda n: mg.image_center + mg.vectors[1]*n

line_100_0 = gl.Line(k(2), k(-2), color="red", width=1)
line_100_1 = gl.Line(k(2) + mg.vectors[0], k(-2) + mg.vectors[0], color="red", width=1)
dist_100 = gl.Arrow(
    line_100_0.pointA, 
    line_100_0.pointA + np.array([mg.vectors[0,0]*np.cos(mg.angle_between_vectors), mg.vectors[0,1]*np.sin(mg.angle_between_vectors)]), 
    two_sided=True, width=1, color="red"
)
print(np.array([mg.vectors[0,0]*np.cos(mg.angle_between_vectors), mg.vectors[0,1]*np.sin(mg.angle_between_vectors)]))

fig.add_elements(
    mg.fft_plot,
    line_100_0,
    line_100_1,
    *mg.arrows
)
fig.show()